# OpenAI API - Function Calling (함수 호출)

Function Calling(함수 호출)은 LLM이 외부 도구(API, 함수 등)를 사용할 수 있도록 연결해주는 기능입니다.
모델은 사용자의 질문을 분석하여 **'어떤 함수를 어떤 인자로 호출해야 하는지'** 판단하고, 그 결과를 JSON 형태로 반환합니다.

**학습 목표:**
1. **Function Calling 흐름 이해:** LLM이 함수 호출을 요청하고, 실행 결과를 받아 최종 답변을 생성하는 과정 이해하기
2. **도구(Tool) 정의:** LLM이 인식할 수 있도록 함수와 파라미터 스키마 정의하기
3. **자동화 루프 구현:** 사용자 질문 -> 함수 호출 판단 -> 함수 실행 -> 결과 반영 -> 최종 답변 생성의 전체 파이프라인 구현하기

① Tool Definitions + Messages  
개발자가 사용할 수 있는 함수(예: get_weather(location))를 미리 정의  
사용자가 질문: “What’s the weather in Paris?”

② Tool Calls  
모델이 질문을 보고 “아, 이건 get_weather("paris") 함수를 호출해야겠네”  
텍스트가 아니라 함수 호출 요청(JSON) 을 생성

③ Execute Function Code  
실제 코드에서 get_weather("paris") 실행  
외부 API(OpenWeather 같은) 호출

④ Results (All Prior Messages)  
함수 실행 결과가 다시 모델에게 전달  
모델은 이제 “파리의 온도 = 14도”라는 사실을 알게 됨

⑤ Final Response  
모델이 사용자에게 자연어로 최종 답변 생성  
“It’s currently 14°C in Paris.”

LLM이 API를 직접 실행하는 게 아니라  
“어떤 함수를 호출할지 결정”만 하고  
실행은 개발자 코드,  
결과를 다시 받아 문장 생성  
👉 LLM + 외부 시스템 연동 구조

In [1]:
from openai import OpenAI
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
OPENWHEATHER_API_KEY = os.getenv("openweather_key")     # .env의 openweather_key 값을 가져옴

In [2]:
# OpenWeather API 호출로 부산 날씨 데이터 조회
import requests

city_name = "Busan"    # 조회할 도시
units = "metric"       # 단위 (metric = 섭씨)
# OpenWeather 현재날씨 API URL 생성
url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWHEATHER_API_KEY}&units={units}"
response = requests.get(url)  # API 요청 전송
data = response.json()        # 응답(JSON)을 파이썬 dict로 변환
data

{'coord': {'lon': 129.0403, 'lat': 35.1028},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'base': 'stations',
 'main': {'temp': -2.01,
  'feels_like': -6.44,
  'temp_min': -2.01,
  'temp_max': -2.01,
  'pressure': 1030,
  'humidity': 43,
  'sea_level': 1030,
  'grnd_level': 1025},
 'visibility': 10000,
 'wind': {'speed': 3.6, 'deg': 310},
 'clouds': {'all': 0},
 'dt': 1770079298,
 'sys': {'type': 1,
  'id': 8086,
  'country': 'KR',
  'sunrise': 1770070906,
  'sunset': 1770108802},
 'timezone': 32400,
 'id': 1838524,
 'name': 'Busan',
 'cod': 200}

In [3]:
weather_info = {}  # 최종 결과를 담을 딕셔너리

if response.status_code == 200:    # 응답 코드 200이면 정상 처리
    weather_description = data['weather'][0]['description']  # 날씨 설명
    temp = data['main']['temp']                              # 온도
    temp_feels_like = data['main']['feels_like']             # 체감 온도
    humidity = data['main']['humidity']                      # 습도

    weather_info = {
        'city': city_name,
        'description': weather_description,
        'temperature': temp,
        'temperature_feels_like': temp_feels_like,
        'humidity': humidity
    }
else:  # 정상 응답이 아니면 실패 처리
    weather_info = {
        'city': city_name,
        'description': 'Not Found',
        'temperature': 'Not Found',
        'temperature_feels_like': 'Not Found',
        'humidity': 'Not Found'
    }

weather_info

{'city': 'Busan',
 'description': 'clear sky',
 'temperature': -2.01,
 'temperature_feels_like': -6.44,
 'humidity': 43}

In [4]:
# 실패시 실패 원인 확인용 디버깅
print("status_code :", response.status_code)  # HTTP 상태코드 (200=성공, 401=키 문제 등)
print("response json :", data)                # 응답받은 JSON
print("message :", data.get("message", "no message"))  # 에러 메시지 키

status_code : 200
response json : {'coord': {'lon': 129.0403, 'lat': 35.1028}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': -2.01, 'feels_like': -6.44, 'temp_min': -2.01, 'temp_max': -2.01, 'pressure': 1030, 'humidity': 43, 'sea_level': 1030, 'grnd_level': 1025}, 'visibility': 10000, 'wind': {'speed': 3.6, 'deg': 310}, 'clouds': {'all': 0}, 'dt': 1770079298, 'sys': {'type': 1, 'id': 8086, 'country': 'KR', 'sunrise': 1770070906, 'sunset': 1770108802}, 'timezone': 32400, 'id': 1838524, 'name': 'Busan', 'cod': 200}
message : no message


In [10]:
# 날씨 API 호출 함수: doc_string을 활용해 함수 설명
import requests
import json

def get_current_weather(city_name='Seoul', units='metric'):
    """
    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    """
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={OPENWHEATHER_API_KEY}&units={units}"
    response = requests.get(url)  # API 요청 전송
    data = response.json()        # 응답(JSON)을 파이썬 dict로 변환
    
    weather_info = {}  # 최종 결과를 담을 딕셔너리

    if response.status_code == 200:    # 응답 코드 200이면 정상 처리
        weather_description = data['weather'][0]['description']  # 날씨 설명
        temp = data['main']['temp']                              # 온도
        temp_feels_like = data['main']['feels_like']             # 체감 온도
        humidity = data['main']['humidity']                      # 습도

        weather_info = {
            'city': city_name,
            'description': weather_description,
            'temperature': temp,
            'temperature_feels_like': temp_feels_like,
            'humidity': humidity
        }
    else:  # 정상 응답이 아니면 실패 처리
        weather_info = {
            'city': city_name,
            'description': 'Not Found',
            'temperature': 'Not Found',
            'temperature_feels_like': 'Not Found',
            'humidity': 'Not Found'
        }
    
    return json.dumps(weather_info)  # dict를 JSON 문자열로 변환해 반환

In [11]:
# llm이 사용할 함수(tool) 모음 
tools_to_execute = {
    'get_current_weather': get_current_weather,  # 키: LLM이 호출할 tool 이름, 값: 실제 실행될 함수
}

## llm 준비

In [12]:
# get_current_weather 함수의 docstring(설명) 출력
print(tools_to_execute['get_current_weather'].__doc__)


    OpenWeather API를 사용해서 사용자가 지정한 도시의 현재 날씨 정보를 가져오는 함수

    Args:
        - city: str 날씨정보를 가져올 도시 이름. **반드시 영문으로 작성하세요.**
            - 변환예시:
                - 서울 -> Seoul
                - 충남, 충청남도 -> Chungcheongnam-do
                - 부산 -> Busan
        - units: str 온도단위를 설정하는 문자열
            - metric(기본값: 섭씨, 미터)
            - imperial(화씨, 야드)
    Return:
        - str: json 형식으로 변환된 현재 날씨 정보
    


In [ ]:
# 사용자 질문을 받아 tool 호출 여부를 판단하고, 필요시 함수를 실행한 뒤 최종 답변을 생성하는 메인 함수
def run_conversation(user_prompt, model='gpt-4.1-mini'):
    # 1. 대화 히스토리 초기화 (시스템 프롬프트 + 사용자 질문)
    messages = [
        {'role': 'system', 'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 후 대답하세요.'},
        {'role': 'user', 'content': user_prompt}
    ]
    
    # 2. LLM이 사용 가능한 함수(Tool) 정의
    # 이 스키마를 통해 LLM은 함수의 기능과 필요한 인자(매개변수)를 이해합니다.
    tools = [
        {
            'type': 'function',
            'function': {
                'name': 'get_current_weather',  # 함수 이름
                'description': tools_to_execute['get_current_weather'].__doc__,  # 함수 설명 (docstring 활용)
                'parameters': {   # JSON Schema 형식으로 파라미터 정의
                    'type': 'object',
                    'properties': {
                        'city_name': {
                            'type': 'string',
                            'description': '''도시이름(필수값). 반드시 영어로 작성하세요.
                                - 변환예시:
                                    - 서울 -> Seoul
                                    - 부산 -> Busan
                            ''' 
                        },
                        'units': {
                            'type': 'string',
                            'description': '온도단위 (metric: 섭씨, imperial: 화씨)',
                            'enum': ['metric', 'imperial']
                        }
                    },
                    'required': ['city_name']  # 필수 파라미터 지정
                }
            }
        }
    ]

    # 3. 1차 LLM 호출 (함수 호출 필요 여부 판단)
    response1 = client.chat.completions.create(
        model = model,
        messages = messages,
        tools = tools    # 정의한 도구 리스트 전달
    )

    response1_message = response1.choices[0].message
    response1_tool_calls = response1_message.tool_calls   # LLM이 함수 호출을 요청했는지 확인

    # 4. 함수 호출 요청이 있는 경우
    if response1_tool_calls:
        # 기존 메시지 히스토리에 LLM의 응답(함수 호출 요청)을 추가
        messages.append(response1_message)

        # 여러 개의 함수 호출 요청이 있을 수 있으므로 반복문 처리
        for tool_call in response1_tool_calls:
            function_name = tool_call.function.name
            print(f'[tool] {function_name}을 호출합니다!')
            
            # 실제 파이썬 함수 실행
            function_to_execute = tools_to_execute[function_name]
            function_args = json.loads(tool_call.function.arguments)  # JSON 문자열을 딕셔너리로 변환
            function_response = function_to_execute(**function_args)  # 함수 실행 및 결과 반환

            # 5. 함수 실행 결과를 메시지 히스토리에 추가 (role='tool')
            messages.append({
                'role': 'tool',
                'tool_call_id': tool_call.id,  # 어떤 호출에 대한 응답인지 매핑
                'name': function_name,
                'content': function_response   # 함수 실행 결과 (보통 JSON 문자열)
            })
            pprint(messages)  # 디버깅: 현재까지의 대화 흐름 출력
        
        # 6. 2차 LLM 호출 (함수 실행 결과를 바탕으로 최종 답변 생성)
        response2 = client.chat.completions.create(
            model = model,
            messages = messages
        )
        return response2.choices[0].message.content
    
    # 7. 함수 호출이 필요 없는 경우 (일반 대화)
    else:
        return response1_message.content

In [18]:
run_conversation("오늘 서울 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 서울 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_es9BY8c9joVSt1sGuimPrHrS', function=Function(arguments='{"city_name":"Seoul","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Seoul", "description": "clear sky", "temperature": '
             '-4.24, "temperature_feels_like": -4.24, "humidity": 68}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_es9BY8c9joVSt1sGuimPrHrS'}]


'오늘 서울은 맑은 하늘이 예상되며, 기온은 약 -4.2도입니다. 체감 온도도 비슷하게 약 -4.2도이고, 습도는 68%입니다. 외출하실 때 따뜻하게 입으시는 것이 좋겠습니다.'

In [19]:
run_conversation("오늘 전남 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 전남 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_u2NMkoRf4Aekvfhipbnw1Gr6', function=Function(arguments='{"city_name":"Jeollanam-do","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Jeollanam-do", "description": "clear sky", '
             '"temperature": 0.83, "temperature_feels_like": -2.61, '
             '"humidity": 54}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_u2NMkoRf4Aekvfhipbnw1Gr6'}]


'오늘 전남은 맑은 하늘에 기온은 약 0.8도이며, 체감 온도는 -2.6도 정도로 다소 쌀쌀합니다. 습도는 54%입니다. 외출 시 참고하세요!'

In [20]:
run_conversation("오늘 오사카 날씨는 어때?")

[tool] get_current_weather을 호출합니다!
[{'content': '당신은 친절한 챗봇입니다. 사용자의 요구를 분석해 직접 대답하거나, 주어진 함수를 이용해 필요한 정보를 먼저 확보한 '
             '후 대답하세요.',
  'role': 'system'},
 {'content': '오늘 오사카 날씨는 어때?', 'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_mnxRRLz4kkkGGZ6GpMLctVzv', function=Function(arguments='{"city_name":"Osaka","units":"metric"}', name='get_current_weather'), type='function')]),
 {'content': '{"city": "Osaka", "description": "few clouds", "temperature": '
             '4.56, "temperature_feels_like": 1.25, "humidity": 60}',
  'name': 'get_current_weather',
  'role': 'tool',
  'tool_call_id': 'call_mnxRRLz4kkkGGZ6GpMLctVzv'}]


'오늘 오사카 날씨는 대체로 구름이 조금 낀 상태이며, 기온은 약 4.6도, 체감 온도는 1.3도 정도입니다. 습도는 60%입니다. 따뜻하게 입으시고 외출하세요!'

In [24]:
run_conversation("독산역 근처 맛집 몇개만 추천해줘봐")

'독산역 근처 맛집을 추천해드릴게요. 어떤 종류의 음식을 원하시나요? 한식, 중식, 일식, 양식, 카페 등 선호하시는 음식 종류나 분위기가 있으면 알려주세요!'